In [2]:
import spacy
import re
from skweak import heuristics, gazetteers, aggregation, utils

In [3]:
def money_detector(doc):
    for tok in doc[1:]:
        if tok.text[0].isdigit() and tok.nbor(-1).is_currency:
            yield tok.i-1, tok.i+1, "MONEY"

In [4]:
lf1 = heuristics.FunctionAnnotator("money", money_detector)

In [5]:
lf2 = heuristics.TokenConstraintAnnotator(
    "years", lambda tok: re.match("(19|20)\d{2}$", tok.text), "DATE")


In [22]:
names = [("Artur", "Zorron"), ("Vitor", "Araruna"), ("Victor", "Vasconcelos"), ("Lucelia", "Vieira")]
trie = gazetteers.Trie(names)
lf3 = gazetteers.GazetteerAnnotator("knedle-team", {"PERSON": trie})

In [23]:
# bands = ["DOPE LEMON", "Tash Sultana", "Ocean Alley"]
# trie = gazetteers.Trie(bands)
# lf4 = gazetteers.GazetteerAnnotator("good-bands", {"BAND": trie})

In [24]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Artur Zorron está querendo comprar um ingresso do show do DOPE LEMON que ocorrerá em Madri em março de 2022 O ingresso custa $100,00")

In [25]:
doc = lf3(lf2(lf1(doc)))

In [28]:
hmm = aggregation.HMM("hmm", ["PERSON", "DATE", "MONEY"])
hmm.fit_and_aggregate([doc])

Starting iteration 1
Finished E-step with 1 documents
Starting iteration 2
Finished E-step with 1 documents
Starting iteration 3
Finished E-step with 1 documents
Starting iteration 4
Finished E-step with 1 documents
Starting iteration 5
Finished E-step with 1 documents


         1         -15.5631             +nan
         2         -15.4148          +0.1483
         3         -15.1820          +0.2328
         4         -14.9440          +0.2380
         5         -14.7952          +0.1489


[Artur Zorron está querendo comprar um ingresso do show do DOPE LEMON que ocorrerá em Madri em março de 2022 O ingresso custa $100,00]

In [29]:
utils.display_entities(doc, "hmm")